In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from zipfile import ZipFile
import requests as r
from io import BytesIO
from geopandas.tools import sjoin
import pandas_bokeh
import geopandas as gpd
from bokeh.util.warnings import BokehDeprecationWarning
import warnings

In [ ]:
pandas_bokeh.output_notebook()

In [ ]:
warnings.simplefilter('ignore', BokehDeprecationWarning)

In [ ]:
# license: https://www.swisstopo.admin.ch/en/home/meta/conditions/geodata/free-geodata.html
boundaries_url = 'http://data.geo.admin.ch/ch.swisstopo.swissboundaries3d-kanton-flaeche.fill/data.zip'
names_url = 'http://data.geo.admin.ch/ch.swisstopo.swissnames3d/data.zip'
npas_url = 'https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_SHP_LV95.zip'
npas_names_url = 'https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_CSV_WGS84.zip'

In [ ]:
def unzip(uri, output=None):
    z = ZipFile(uri)
    z.extractall(output)

In [ ]:
def load(url):
    req = r.get(url, stream=True)
    unzip(BytesIO(req.content), '../data')

In [ ]:
load(boundaries_url)

In [ ]:
load(names_url)

In [ ]:
load(npas_url)

In [ ]:
load(npas_names_url)

In [ ]:
unzip('../data/SHAPEFILE_LV95_LN02.zip', '../data')

In [ ]:
unzip('../data/swissNAMES3D_LV95.zip', '../data')

In [ ]:
# Pays / Land
countries_uri = '../data/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_LANDESGEBIET.shp'
countries = gpd.read_file(countries_uri)
countries.crs = {'init': 'epsg:2056'}
countries = countries.to_crs(epsg=4326)

# filter DE/IT/LI
countries = countries[countries.ICC == 'CH']

countries = countries[['UUID', 'ICC', 'NAME', 'LANDESFLAE', 'SEE_FLAECH', 'EINWOHNERZ', 'geometry']]

In [ ]:
countries.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
countries.head()

In [ ]:
#countries[['geometry']].to_file('../out/countries.geojson', driver='GeoJSON')

In [ ]:
# Cantons / Kantone
regions_uri = '../data/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp'
regions = gpd.read_file(regions_uri)
regions.crs = {'init': 'epsg:2056'}
regions = regions.to_crs(epsg=4326)

regions = regions[['UUID', 'NAME', 'SEE_FLAECH', 'KANTONSNUM', 'KANTONSFLA', 'KT_TEIL', 'EINWOHNERZ', 'geometry']].sort_values('KT_TEIL').dissolve('KANTONSNUM').reset_index().drop(columns='KT_TEIL')

In [ ]:
regions.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
regions.head()

In [ ]:
#regions[['geometry']].to_file('../out/regions.geojson', driver='GeoJSON')

In [ ]:
# Districts / Bezirke
counties_url = '../data/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_BEZIRKSGEBIET.shp'
counties = gpd.read_file(counties_url)
counties.crs = {'init': 'epsg:2056'}
counties = counties.to_crs(epsg=4326)

counties = counties[['UUID', 'NAME', 'SEE_FLAECH', 'BEZIRKSNUM', 'BEZIRKSFLA', 'BEZIRK_TEI', 'KANTONSNUM', 'EINWOHNERZ', 'geometry']].sort_values('BEZIRK_TEI').dissolve('BEZIRKSNUM').reset_index().drop(columns='BEZIRK_TEI')

In [ ]:
counties.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
counties.head()

In [ ]:
#counties[['geometry']].to_file('../out/counties.geojson', driver='GeoJSON')

In [ ]:
# communes / Gemeinden
localadmins_uri = '../data/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp'
localadmins = gpd.read_file(localadmins_uri)
localadmins.crs = {'init': 'epsg:2056'}
localadmins = localadmins.to_crs(epsg=4326)

# filter DE/IT/LI
localadmins = localadmins[localadmins.ICC == 'CH']

localadmins = localadmins[['UUID', 'ICC', 'NAME', 'SEE_FLAECH', 'SHN', 'GEM_FLAECH', 'GEM_TEIL', 'BFS_NUMMER', 'KANTONSNUM', 'EINWOHNERZ', 'geometry']].sort_values('GEM_TEIL').dissolve('BFS_NUMMER').reset_index().drop(columns='GEM_TEIL')

In [ ]:
localadmins.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
localadmins.head()

In [ ]:
#localadmins[['geometry']].to_file('../out/localadmins.geojson', driver='GeoJSON')

In [ ]:
# Villes+villages / Städte+Dörfer
localities_uri = '../data/swissNAMES3D_LV95/shp_LV95_LN02/swissNAMES3D_PLY.shp'
localities = gpd.read_file(localities_uri)
localities.crs = {'init': 'epsg:2056'}
localities = localities.to_crs(epsg=4326)

# filter fremd (bigs regions) + k_W (schools, lakes, etc.)
localities = localities[(localities.STATUS != 'fremd') & (localities.STATUS != 'k_WOBJEKTART')]
# keep places
localities = localities[localities.OBJEKTART == 'Ort']
# only in CH
localities = sjoin(countries[['geometry']], localities)

localities = localities[['UUID', 'NAME', 'SPRACHCODE', 'geometry']]

In [ ]:
localities.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
localities.head()

In [ ]:
# slow-slow, heavy-heavy
#localities[['geometry']].to_file('../out/localities.geojson', driver='GeoJSON')

In [ ]:
# Quartiers / Nachbarschaften
neighborhoods_uri = '../data/swissNAMES3D_LV95/shp_LV95_LN02/swissNAMES3D_PLY.shp'
neighborhoods = gpd.read_file(neighborhoods_uri)
neighborhoods.crs = {'init': 'epsg:2056'}
neighborhoods = neighborhoods.to_crs(epsg=4326)

# filter fremd (bigs regions) + k_W (schools, lakes, etc.)
neighborhoods = neighborhoods[(neighborhoods.STATUS != 'fremd') & (neighborhoods.STATUS != 'k_WOBJEKTART')]
# keep neighborhood parts
neighborhoods = neighborhoods[neighborhoods.OBJEKTART == 'Quartierteil']
# only in CH
neighborhoods = sjoin(countries[['geometry']], neighborhoods)

neighborhoods = neighborhoods[['UUID', 'NAME', 'SPRACHCODE', 'geometry']]

In [ ]:
neighborhoods.plot_bokeh(simplify_shapes=100, hovertool_string="@NAME")
neighborhoods.head()

In [ ]:
# slow-slow, heavy-heavy
#neighborhoods[['geometry']].to_file('../out/neighborhoods.geojson', driver='GeoJSON')

In [ ]:
npas_uri = '../data/PLZO_SHP_LV95/PLZO_PLZ.shp'
npas = gpd.read_file(npas_uri)
npas.crs = {'init': 'epsg:2056'}
npas = npas.to_crs(epsg=4326)
npas = npas[['PLZ', 'ZUSZIFF', 'geometry']]

In [ ]:
npas_name_uri = '../data/PLZO_CSV_WGS84/PLZO_CSV_WGS84.csv'
npas_name = pd.read_csv(npas_name_uri, sep=';', encoding="ISO-8859-1")
npas_name = npas_name.drop(['E', 'N'], axis=1).drop_duplicates()
npas_name = npas_name[~npas_name.duplicated(['PLZ', 'Zusatzziffer'], keep='first')]
npas_name = npas_name.set_index(['PLZ', 'Zusatzziffer'])

In [ ]:
# Code postaux / Postleitzahlen
postal_codes = npas.assign(
    locality=npas[['PLZ', 'ZUSZIFF']].apply(lambda x: npas_name.loc[x.PLZ, x.ZUSZIFF].Ortschaftsname, axis=1),
    localadmin=npas[['PLZ', 'ZUSZIFF']].apply(lambda x: npas_name.loc[x.PLZ, x.ZUSZIFF].Gemeindename, axis=1)
)
postal_codes = postal_codes[['locality', 'localadmin', 'PLZ', 'geometry']]
postal_codes = postal_codes.rename({'PLZ': 'zip'}, axis=1)

In [ ]:
postal_codes.plot_bokeh(simplify_shapes=100, hovertool_string="@PLZ @locality (@localadmin)")
postal_codes.head()

In [ ]:
postal_codes[['geometry']].to_file('../out/postal_codes.geojson', driver='GeoJSON')